# Выбор локации для скважины

Для добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. А также проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Содержание

1. Загрузка и подготовка данных 
2. Обучение и проверка модели
3. Подготовка к расчёту прибыли
4. Расчёт прибыли и рисков

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump
pd.options.mode.chained_assignment = None
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Загрузка и подготовка данных

In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
display(data1.head(10))
data1.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
# посмотрим на основные статистические показатели
# с помощью метода .describe()
data1.describe().round(2)

,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,0.50,0.25,2.50,92.50
std,0.87,0.50,3.25,44.29
min,-1.41,-0.85,-12.09,0.00
25%,-0.07,-0.20,0.29,56.50
50%,0.50,0.25,2.52,91.85
75%,1.07,0.70,4.72,128.56
max,2.36,1.34,16.00,185.36


In [5]:
display(data2.head(10))
data2.info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
# посмотрим на основные статистические показатели
# с помощью метода .describe()
data2.describe().round(2)

,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,1.14,-4.80,2.49,68.83
std,8.97,5.12,1.70,45.94
min,-31.61,-26.36,-0.02,0.00
25%,-6.30,-8.27,1.00,26.95
50%,1.15,-4.81,2.01,57.09
75%,8.62,-1.33,4.00,107.81
max,29.42,18.73,5.02,137.95


In [7]:
display(data3.head(10))
data3.info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
# посмотрим на основные статистические показатели
# с помощью метода .describe()
data3.describe().round(2)

,f0,f1,f2,product
count,100000.00,100000.00,100000.00,100000.00
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.60
max,7.24,7.84,16.74,190.03


Далее посчитаем коэффициент корреляции для датафреймов.

In [9]:
corr_matrix1 = data1.corr(numeric_only=True).round(2)
corr_matrix1

,f0,f1,f2,product
f0,1.00,-0.44,-0.00,0.14
f1,-0.44,1.00,0.00,-0.19
f2,-0.00,0.00,1.00,0.48
product,0.14,-0.19,0.48,1.00


In [10]:
corr_matrix2 = data2.corr(numeric_only=True).round(2)
corr_matrix2

,f0,f1,f2,product
f0,1.00,0.18,-0.0,-0.03
f1,0.18,1.00,-0.0,-0.01
f2,-0.00,-0.00,1.0,1.00
product,-0.03,-0.01,1.0,1.00


In [11]:
corr_matrix3 = data3.corr(numeric_only=True).round(2)
corr_matrix3

,f0,f1,f2,product
f0,1.0,0.0,-0.00,-0.00
f1,0.0,1.0,0.00,-0.00
f2,-0.0,0.0,1.00,0.45
product,-0.0,-0.0,0.45,1.00


Первый вывод, который можно сделать после загрузки данных, что все 3 датафрейма по сути идентичны друг другу. Столбец `id` никак не поможет нам в исследовании, от него можно и нужно избавиться, но перед этим найдем явные дубликаты (если есть совпадение по ID, от них точно надо избавиться). 

In [12]:
data1['id'] = data1['id'].drop_duplicates()

In [13]:
data1['id'] = data1['id'].drop_duplicates()

In [14]:
data1['id'] = data1['id'].drop_duplicates()

In [15]:
data1 = data1.drop(['id'], axis=1)

In [16]:
data2 = data2.drop(['id'], axis=1)

In [17]:
data3 = data3.drop(['id'], axis=1)

Второе - интересно, что в первом датафрейме коэффициенты корреляции варьируются в значениях от -0.44 до 0.48. Во втором и третьем фрейме данных ситуация совсем другая. Во втором фрейме от f2 напрямую зависит добыча нефти. В третьем случае по нулям для f0 и f1, f2 же снова оказывает влияние на результаты добычи.

Теперь нужно разбить данные по 2 выборки

- обучающая выборка
- валидационная выборка

In [18]:
features1 = data1.drop(['product'], axis=1)
target1 = data1['product']

features1_train, features1_valid, target1_train, target1_valid = train_test_split(features1, target1, 
                                                                          test_size=0.25, random_state=12345)
print('Размеры обучающей выборки:')
print(features1_train.shape)
print(target1_train.shape)
print()
print('Размеры валидационной выборки:')
print(features1_valid.shape)
print(target1_valid.shape)
print()

Размеры обучающей выборки:
(75000, 3)
(75000,)

Размеры валидационной выборки:
(25000, 3)
(25000,)



In [19]:
features2 = data2.drop(['product'], axis=1)
target2 = data2['product']

features2_train, features2_valid, target2_train, target2_valid = train_test_split(features2, target2, 
                                                                          test_size=0.25, random_state=12345)
print('Размеры обучающей выборки:')
print(features2_train.shape)
print(target2_train.shape)
print()
print('Размеры валидационной выборки:')
print(features2_valid.shape)
print(target2_valid.shape)
print()

Размеры обучающей выборки:
(75000, 3)
(75000,)

Размеры валидационной выборки:
(25000, 3)
(25000,)



In [20]:
features3 = data3.drop(['product'], axis=1)
target3 = data3['product']

features3_train, features3_valid, target3_train, target3_valid = train_test_split(features3, target3, 
                                                                          test_size=0.25, random_state=12345)
print('Размеры обучающей выборки:')
print(features3_train.shape)
print(target3_train.shape)
print()
print('Размеры валидационной выборки:')
print(features3_valid.shape)
print(target3_valid.shape)
print()

Размеры обучающей выборки:
(75000, 3)
(75000,)

Размеры валидационной выборки:
(25000, 3)
(25000,)



Мы получили по 2 выборки для каждого фрейма данных в пропорциях 75/25. Обучающая выборка забрала 75% данных, валидационная заняла оставшиеся 25%.

Для начала создадим 2 списка, в которых сохраним предсказанные и правиьные ответы.

In [21]:
predicts = []
answers = []

In [22]:
# создаем объект класса LinearRegression и запишем в переменную model
model = LinearRegression()

#обучим нашу модель
model.fit(features1_train, target1_train)

LinearRegression()

In [23]:
# на основе независимых переменных предскажем добычу нефти
predict1 = model.predict(features1_valid)

# выведем первые 5 значения с помощью диапазона индексов
predict1[:5]

array([95.89495185, 77.57258261, 77.89263965, 90.17513418, 70.51008829])

In [24]:
# выведем корень среднеквадратической ошибки
# сравним тестовые и прогнозные значения
print('RMSE:', np.sqrt(mean_squared_error(target1_valid, predict1)))

# выведем средний запас предсказанного сырья 
predict1.mean().round(2)

RMSE: 37.5794217150813


92.59

In [25]:
# посмотрим на еще одну метрику, R2
print('R2:', np.round(r2_score(target1_valid, predict1), 2))

R2: 0.28


In [26]:
predicts.append(predict1)
answers.append(target1_valid)

Повторим работу на других датафреймах.

In [27]:
# создаем объект класса LinearRegression и запишем в переменную model
model = LinearRegression()

#обучим нашу модель
model.fit(features2_train, target2_train)

LinearRegression()

In [28]:
# на основе независимых переменных предскажем добычу нефти
predict2 = model.predict(features2_valid)

# выведем первые 5 значения с помощью диапазона индексов
predict1[:5]

array([95.89495185, 77.57258261, 77.89263965, 90.17513418, 70.51008829])

In [29]:
# выведем корень среднеквадратической ошибки
# сравним тестовые и прогнозные значения
print('RMSE:', np.sqrt(mean_squared_error(target2_valid, predict2)))

# выведем средний запас предсказанного сырья 
predict2.mean().round(2)

RMSE: 0.8930992867756168


68.73

In [30]:
# посмотрим на еще одну метрику, R2
print('R2:', np.round(r2_score(target2_valid, predict2), 2))

R2: 1.0


In [31]:
predicts.append(predict2)
answers.append(target2_valid)

Вторая модель практически идеально предсказывает, это можно объяснить максимальной корреляцией между f2 и добычей.

In [32]:
# создаем объект класса LinearRegression и запишем в переменную model
model = LinearRegression()

#обучим нашу модель
model.fit(features3_train, target3_train)

LinearRegression()

In [33]:
# на основе независимых переменных предскажем добычу нефти
predict3 = model.predict(features3_valid)

# выведем первые 5 значения с помощью диапазона индексов
predict1[:5]

array([95.89495185, 77.57258261, 77.89263965, 90.17513418, 70.51008829])

In [34]:
# выведем корень среднеквадратической ошибки
# сравним тестовые и прогнозные значения
print('RMSE:', np.sqrt(mean_squared_error(target3_valid, predict3)))

# выведем средний запас предсказанного сырья 
predict3.mean().round(2)

RMSE: 40.02970873393434


94.97

In [35]:
# посмотрим на еще одну метрику, R2
print('R2:', np.round(r2_score(target3_valid, predict3), 2))

R2: 0.21


In [36]:
predicts.append(predict3)
answers.append(target3_valid)

Третья модель предсказывает еще чуть хуже чем первая.

Можно подметить, что несмотря на высокую точность предсказания второй модели, средний запас добываемого сырья (предсказанного) у нее гораздо ниже (68.73 против 92.59 и 94.97).

In [37]:
predict1 = pd.Series(predict1)
predict2 = pd.Series(predict2)
predict3 = pd.Series(predict3)

In [38]:
#target1_valid = target1_valid.reset_index(drop=True)
#target2_valid = target2_valid.reset_index(drop=True)
#target3_valid = target3_valid.reset_index(drop=True)

Все ключевые значения для расчётов сохраним в отдельных переменных.

## Подготовка к расчёту прибыли

Посчитаем минимальное среднее количество продукта в месторождениях региона, достаточное для разработки.

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

По условию задачи бюджет на разработку скважин в регионе — 10 млрд рублей. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [39]:
#сначала для ключевых значений создадим константы
PRICE_FOR_1000B = 450000
EXPENSES = 10_000_000_000
LOSS_THRESHOLD = 2.5

# для рассчета минимального среднего разделим финансирование 
# на "выхлоп" с 1 добытой единицы

min_production = 10000000000 / 450000
min_production

22222.222222222223

In [40]:
# теперь полученное значение разделим на количество скважин
# это 200 скважин, лучших для разработки
min_production / 200

111.11111111111111

Теперь сравним это значение со средним количеством сырья в скважине каждого региона.

In [41]:
data1['product'].mean().round(2)

92.5

In [42]:
data2['product'].mean().round(2)

68.83

In [43]:
data3['product'].mean().round(2)

95.0

Подытожим: для того, чтобы разработка новых скважин была безубыточна, среднее количество сырья в скважине должно превышать 111 единиц. Ни в одном регионе среднее количество сырья не превышает необходимое значение. Значит из 100000 вариантов нужно выбрать лучшие скважины, количество сырья в которых будет превышать среднее значение для безубыточной добычи.

## Расчёт прибыли и рисков 

Напишем функцию расчета прибыли. Она будет:
1. Выбирать скважины с максимальными значениями предсказаний. 
2. Суммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
3. Рассчитывать прибыль для полученного объёма сырья.

In [44]:
def income(predictions, target):
    preds_sorted = predictions.sort_values(ascending=False)[:200]       
    selected = target[preds_sorted.index]   
    return PRICE_FOR_1000B * selected.sum() - EXPENSES

Далее посчитаем риски и прибыль для каждого региона.

In [46]:
# создадим функцию чтобы далее не пришлось дублировать весь этот код
# в функции применим технику Bootstrap с 1000 выборок

def calculations (target, predict):    
    state = np.random.RandomState(12345)
    values = []
    lost = 0  
    target = target.reset_index(drop=True)
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predict[target_subsample.index]
        values.append(income(preds_subsample, target))
        
    values = pd.Series(values)    
    
    # узнаем риски
    risk = (values < 0).mean() * 100        
        
    # найдем среднюю прибыль       
    mean = values.mean()  
    print("Средняя прибыль:", mean.round(2))
    
    # узнаем 95% доверительный интервал
    conf_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    print("95%-ый доверительный интервал:", conf_interval)
    
    if risk < LOSS_THRESHOLD:
        print(f'Вероятность убытков - {risk}%, меньше допустимой вероятности {LOSS_THRESHOLD}%')
    else:
        print(f'Вероятность убытков - {risk}%, больше допустимой вероятности {LOSS_THRESHOLD}%')    
    

In [47]:
calculations (target1_valid, predict1)

Средняя прибыль: 396164984.8
95%-ый доверительный интервал: (379620315.1479725, 412709654.45676965)
Вероятность убытков - 6.9%, больше допустимой вероятности 2.5%


Средняя прибыль в регионе планируется в 396.164.984,8 рублей. С 95% вероятностью разброс выручки составляет 33млн рублей. Вероятность убытков - 6.9%, больше допустимой вероятности 2.5%

In [48]:
calculations (target2_valid, predict2)

Средняя прибыль: 456045105.79
95%-ый доверительный интервал: (443147248.66390055, 468942962.909421)
Вероятность убытков - 1.5%, меньше допустимой вероятности 2.5%


Средняя прибыль в регионе планируется в 456.045.105,79 рублей. С 95% вероятностью разброс выручки составляет 25млн рублей. Вероятность убытков - 1.5%, меньше допустимой вероятности 2.5%

In [49]:
calculations (target3_valid, predict3)

Средняя прибыль: 404403866.57
95%-ый доверительный интервал: (387445797.4712804, 421361935.6654332)
Вероятность убытков - 7.6%, больше допустимой вероятности 2.5%


Средняя прибыль в регионе планируется в 404.403.866,57 рублей. С 95% вероятностью разброс выручки составляет 34млн рублей. Вероятность убытков - 7.6%, больше допустимой вероятности 2.5%

**Вывод**: Во втором регионе, судя по результатам исследования, ожидается самая высокая выручка со скважин, а также с вероятностью 95% самая высокая минимальная прибыль.

## Общий вывод

**Мне была поставлена задача от добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.**

В результате ознакомления с данными было выявлено,  что в первом датафрейме коэффициенты корреляции варьируются в значениях от -0.44 до 0.48. Во втором и третьем фрейме данных ситуация совсем другая. Во втором фрейме от f2 напрямую зависит добыча нефти. В третьем случае по нулям для f0 и f1, f2 же снова оказывает влияние на результаты добычи. Это интересный факт.


Первым этапом, после ознакомления с данными, была проведена подготовка данных.
1. Из фреймов был удален столбец `id(уникальный идентификатор скважины)`, который никак не поможет исследованию и никак не влияет на целевой признак.

3. Выборки были разбиты на 2 части:
 - обучающая выборка
 - валидационная выборка


Далее, вторым этапом, было проведено обучение и проверка моделей.


    В результате исследования задачи, были получены результаты среднего запас предсказанного сырья 92.59, 68.73 и 94.97 единиц добычи для 1, 2 и 3 региона соответственно. 
    
На этапе подготовки к расчёту прибыли, было вычисленно, что минимальное среднее количество продукта в месторождениях региона, достаточное для разработки должно быть не менее 111.1 единиц продукта на скважину. **Очевидно, что для того, чтобы разработка новых скважин была безубыточна, среднее количество сырья в скважине должно превышать 111 единиц. Ни в одном регионе среднее количество сырья не превышает необходимое значение. Значит из 100000 вариантов нужно выбрать лучшие скважины (200 скважин), количество сырья в которых будет превышать среднее значение для безубыточной добычи.**


    После расчёта прибыли и рисков для 200 лучших скважин каждого региона, мы получили следующие значения: 
- Для первого региона:
1. Средняя прибыль: 396 млн
2. 95%-ый доверительный интервал: (379, 412)
3. Вероятность убытков - 6.9%, больше допустимой вероятности 2.5%

Средняя прибыль в регионе планируется в 396.164.984,8 рублей. С 95% вероятностью разброс выручки составляет 33млн рублей. Вероятность убытков - 6.9%, больше допустимой вероятности 2.5%


- Для второго региона:
1. Средняя прибыль: 456 млн
2. 95%-ый доверительный интервал: (443, 468)
3. Вероятность убытков - 1.5%, меньше допустимой вероятности 2.5%


Средняя прибыль в регионе планируется в 456.045.105,79 рублей. С 95% вероятностью разброс выручки составляет 25млн рублей. Вероятность убытков - 1.5%, меньше допустимой вероятности 2.5%


- Для третьего региона:
1. Средняя прибыль: 404 млн
2. 95%-ый доверительный интервал: (387, 421)
3. Вероятность убытков - 7.6%, больше допустимой вероятности 2.5%

Средняя прибыль в регионе планируется в 404.403.866,57 рублей. С 95% вероятностью разброс выручки составляет 34млн рублей. Вероятность убытков - 7.6%, больше допустимой вероятности 2.5%

**Во втором регионе, судя по результатам исследования, ожидается самая высокая средняя выручка со скважин, а также самые меньшие риски убытков. Менее всего рекомендуется третий регион, вероятность убытков в нем 7.6%, что больше допустимой вероятности в 2.5%**